## 2 Private Training
Student Name & Number =Sara Rezanezhad / 99101643
Student Name & Number =Kimia Fakheri / *******


## Import necessary packages

In [3]:
!pip install opacus


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np

from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
import torch.nn.functional as F
from opacus import PrivacyEngine

## Provided Model

In [8]:
class CIFAR10Classifier(nn.Module):
  def __init__(self):
    super(CIFAR10Classifier, self).__init__()
    self.conv1 = nn.Conv2d(3, 16, 3, 1)
    self.conv2 = nn.Conv2d(16, 32, 3, 1)
    self.dropout1 = nn.Dropout2d(0.25)
    self.dropout2 = nn.Dropout2d(0.5)
    self.fc1 = nn.Linear(6272, 64)
    self.fc2 = nn.Linear(64, 10)

  def forward(self, x):
    x = self.conv1(x)
    x = F.relu(x)
    x = self.conv2(x)
    x = F.relu(x)
    x = F.max_pool2d(x, 2)
    x = self.dropout1(x)
    x = torch.flatten(x, 1)
    x = self.fc1(x)
    x = F.relu(x)
    x = self.dropout2(x)
    x = self.fc2(x)
    return x

## Baseline Model (Simulation Question 4)
This code trains the CIFAR10Classifier model on 80% of the CIFAR-10 training data and uses the remaining 20% as the validation set. The model is trained for 20 epochs, and the training loss, validation loss, and validation accuracy are printed at the end of each epoch.


In [ ]:
# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Load CIFAR-10 dataset (assuming you've already done this)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Split the dataset into training (80%) and validation (20%) sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False)

# Initialize the model and move it to the device
model = CIFAR10Classifier().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss /= len(val_loader)
    val_accuracy = 100 * correct / total

    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

torch.save(model.state_dict(), 'BaselineModel.pth')


Files already downloaded and verified
Epoch [1/10], Train Loss: 1.7640, Val Loss: 1.4198, Val Accuracy: 50.92%
Epoch [2/10], Train Loss: 1.5207, Val Loss: 1.2703, Val Accuracy: 57.32%
Epoch [3/10], Train Loss: 1.4126, Val Loss: 1.1672, Val Accuracy: 59.51%
Epoch [4/10], Train Loss: 1.3383, Val Loss: 1.1189, Val Accuracy: 61.36%
Epoch [5/10], Train Loss: 1.2799, Val Loss: 1.0770, Val Accuracy: 62.80%
Epoch [6/10], Train Loss: 1.2345, Val Loss: 1.0694, Val Accuracy: 62.75%
Epoch [7/10], Train Loss: 1.1960, Val Loss: 1.0383, Val Accuracy: 64.15%
Epoch [8/10], Train Loss: 1.1653, Val Loss: 1.0301, Val Accuracy: 63.87%
Epoch [9/10], Train Loss: 1.1321, Val Loss: 1.0177, Val Accuracy: 65.03%
Epoch [10/10], Train Loss: 1.1101, Val Loss: 1.0064, Val Accuracy: 65.12%


## Modified model(Simulation Question 5)

In [ ]:
# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the CIFAR-10 dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Split the training data into training and validation sets
num_train = len(trainset)
indices = list(range(num_train))
np.random.shuffle(indices)
train_indices = indices[:int(0.8 * num_train)]
val_indices = indices[int(0.8 * num_train):]
train_subset = Subset(trainset, train_indices)
val_subset = Subset(trainset, val_indices)

# Create data loaders
train_loader = DataLoader(train_subset, batch_size=64, shuffle=True, num_workers=2)
val_loader = DataLoader(val_subset, batch_size=64, shuffle=False, num_workers=2)

# Initialize the model and move it to the device
model2 = CIFAR10Classifier().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model2.parameters(), lr=0.001)

# Initialize the Privacy Engine
privacy_engine = PrivacyEngine()

# Make the model, optimizer, and data loader private
model2, optimizer, train_loader = privacy_engine.make_private(
    module=model2,
    optimizer=optimizer,
    data_loader=train_loader,
    noise_multiplier=0.1,
    max_grad_norm=1.0
)

#privacy_engine.attach(optimizer)

# Train the model
num_epochs = 15
for epoch in range(num_epochs):
    # Training
    model2.train()
    train_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model2(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    train_loss /= len(train_loader)


    # Validation model
    model2.eval()
    val_loss = 0.0
    total = 0
    correct = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model2(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_loss /= len(val_loader)
    val_accuracy = (correct * 100) / total

    # Calculate the privacy budget spent so far
    epsilon = privacy_engine.get_epsilon(delta=1e-5)
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%, Epsilon: {epsilon:.2f}')

torch.save(model2.state_dict(), 'ModifiedModel.pth')


Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/opacus/privacy_engine.py:95: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1373: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Epoch [1/12], Train Loss: 2.0898, Val Loss: 1.8559, Val Accuracy: 34.40%, Epsilon: 64.65
Epoch [2/12], Train Loss: 1.9669, Val Loss: 1.7622, Val Accuracy: 37.67%, Epsilon: 85.01
Epoch [3/12], Train Loss: 1.9361, Val Loss: 1.7044, Val Accuracy: 40.65%, Epsilon: 101.70
Epoch [4/12], Train Loss: 1.9203, Val Loss: 1.6565, Val Accuracy: 42.52%, Epsilon: 116.54
Epoch [5/12], Train Loss: 1.9130, Val Loss: 1.6617, Val Accuracy: 43.34%, Epsilon: 130.23
Epoch [6/12], Train Loss: 1.9132, Val Loss: 1.6240, Val Accuracy: 44.72%, Epsilon: 143.11
Epoch [7/12], Train Loss: 1.9212, Val Loss: 1.6018, Val Accuracy: 45.52%, Epsilon: 155.38
Epoch [8/12], Train Loss: 1.9092, Val Loss: 1.5699, Val Accuracy: 46.80%, Epsilon: 167.16


/usr/local/lib/python3.10/dist-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the smallest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(


Epoch [9/12], Train Loss: 1.9115, Val Loss: 1.6264, Val Accuracy: 46.35%, Epsilon: 178.56
Epoch [10/12], Train Loss: 1.9118, Val Loss: 1.5770, Val Accuracy: 46.91%, Epsilon: 189.64
Epoch [11/12], Train Loss: 1.9074, Val Loss: 1.5631, Val Accuracy: 47.10%, Epsilon: 200.44
Epoch [12/12], Train Loss: 1.9029, Val Loss: 1.5963, Val Accuracy: 48.08%, Epsilon: 211.00


## Training the two attacker models based on MIA techniques (Simulation Question 6)

### Attacker for Modifide Model

In [12]:
# Load the modified model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modified_model = CIFAR10Classifier().to(device)
state_dict = torch.load('/content/ModifiedModel.pth', map_location='cpu')
new_state_dict = {k.replace('_module.', ''): v for k, v in state_dict.items()}  # Adjust keys if needed

# Load the modified state_dict into the model
modified_model.load_state_dict(new_state_dict)
# Get the output size of the baseline model
with torch.no_grad():
    example_input = torch.randn(1, 3, 32, 32).to(device)
    modified_output = modified_model(example_input)
    attacker_modified_input_size = modified_output.size(1)


transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# Load the training and test data
train_data = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_data = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Split the training data into 80% seen data and 20% unseen data
train_seen, train_unseen, _, _ = train_test_split(train_data, train_data, test_size=0.2, random_state=42)

# Create PyTorch DataLoaders for the seen and unseen data
train_seen_loader = DataLoader(train_seen, batch_size=32, shuffle=True)
train_unseen_loader = DataLoader(train_unseen, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define the input and output sizes for the shadow and attacker models
input_size = 3 * 32 * 32  # Assuming CIFAR-10 images with 3 channels and 32x32 resolution
output_size = 10  # 10 classes in CIFAR-10
hidden_size = 128

# Define the shadow model architecture
shadow_model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, output_size)
)

# Train the shadow model on the seen data
optimizer = optim.Adam(shadow_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs =10
for epoch in range(num_epochs):
    for data, target in train_seen_loader:
        optimizer.zero_grad()
        output = shadow_model(data.view(data.size(0), -1))  # Flatten the input data
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

# Train the attacker model on the seen and unseen data
attacker_model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, 10)  # Output 2 classes: member or non-member
)

optimizer = optim.Adam(attacker_model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

for epoch in range(num_epochs):
    for data, target in train_seen_loader:
        optimizer.zero_grad()
        output = attacker_model(data.view(data.size(0), -1))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

    for data, target in train_unseen_loader:
        optimizer.zero_grad()
        output = attacker_model(data.view(data.size(0), -1))
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch}/{num_epochs}] Loss:{loss:.4f}")


def calculate_mia_accuracy(attacker_model, train_seen_loader, train_unseen_loader, test_loader):
    correct_seen = 0
    total_seen = 0
    correct_unseen = 0
    total_unseen = 0

    with torch.no_grad():
        for data, target in train_seen_loader:
            output = attacker_model(data.view(data.size(0), -1))
            pred = output.argmax(dim=1, keepdim=True)
            correct_seen += pred.eq(target.view_as(pred)).sum().item()
            total_seen += data.size(0)

        for data, target in train_unseen_loader:
            output = attacker_model(data.view(data.size(0), -1))
            pred = output.argmax(dim=1, keepdim=True)
            correct_unseen += pred.eq(target.view_as(pred)).sum().item()
            total_unseen += data.size(0)

    seen_accuracy = correct_seen / total_seen
    unseen_accuracy = correct_unseen / total_unseen
    mia_accuracy = (seen_accuracy + unseen_accuracy) / 2

    return mia_accuracy

# Calculate the MIA accuracy
mia_accuracy = calculate_mia_accuracy(attacker_model, train_seen_loader, train_unseen_loader, test_loader)
print(f"MIA Accuracy: {mia_accuracy:.4f}")


Files already downloaded and verified
Files already downloaded and verified
Epoch [0/10] Loss:1.0638
Epoch [1/10] Loss:2.1377
Epoch [2/10] Loss:1.2463
Epoch [3/10] Loss:0.8980
Epoch [4/10] Loss:1.1552
Epoch [5/10] Loss:0.8915
Epoch [6/10] Loss:1.0408
Epoch [7/10] Loss:1.2874
Epoch [8/10] Loss:0.9881
Epoch [9/10] Loss:0.8388
MIA Accuracy: 0.6757


## Attacker for BaseLine

In [13]:
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
import torchvision
import torchvision.transforms as transforms

# Train multiple shadow models
num_shadow_models = 5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load CIFAR-10 dataset (assuming you've already done this)
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

# Split the dataset into training (80%) and validation (20%) sets
train_size = int(0.8 * len(trainset))
val_size = len(trainset) - train_size
trainset, valset = torch.utils.data.random_split(trainset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(valset, batch_size=64, shuffle=False)
# Define the loss function and optimizer


for i in range(num_shadow_models):
    shadow_model = CIFAR10Classifier().to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(shadow_model.parameters(), lr=0.001)
    for epoch in range(3):
        # Training
        shadow_model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = shadow_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_loss /= len(train_loader)

        # Validation
        shadow_model.eval()
        val_loss = 0.0
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = shadow_model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        val_loss /= len(val_loader)
        val_accuracy = 100 * correct / total

        print(f'Epoch [{epoch+1}/{3}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

    # Save the trained shadow model
    torch.save(shadow_model.state_dict(), f'ShadowModel_{i}.pth')


Files already downloaded and verified
Epoch [1/3], Train Loss: 1.7808, Val Loss: 1.4418, Val Accuracy: 49.36%
Epoch [2/3], Train Loss: 1.5333, Val Loss: 1.3025, Val Accuracy: 54.03%
Epoch [3/3], Train Loss: 1.4460, Val Loss: 1.2170, Val Accuracy: 58.14%
Epoch [1/3], Train Loss: 1.7613, Val Loss: 1.4022, Val Accuracy: 50.67%
Epoch [2/3], Train Loss: 1.5051, Val Loss: 1.2442, Val Accuracy: 56.45%
Epoch [3/3], Train Loss: 1.3935, Val Loss: 1.1691, Val Accuracy: 59.91%
Epoch [1/3], Train Loss: 1.7684, Val Loss: 1.3919, Val Accuracy: 50.82%
Epoch [2/3], Train Loss: 1.5011, Val Loss: 1.2453, Val Accuracy: 55.63%
Epoch [3/3], Train Loss: 1.3918, Val Loss: 1.1591, Val Accuracy: 59.94%
Epoch [1/3], Train Loss: 1.7851, Val Loss: 1.4396, Val Accuracy: 49.76%
Epoch [2/3], Train Loss: 1.5516, Val Loss: 1.2998, Val Accuracy: 55.13%
Epoch [3/3], Train Loss: 1.4457, Val Loss: 1.2099, Val Accuracy: 58.71%
Epoch [1/3], Train Loss: 1.7656, Val Loss: 1.4025, Val Accuracy: 51.14%
Epoch [2/3], Train Loss: 1

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import ConcatDataset
from torchvision import datasets, transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the attacker model architecture
class AttackerModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(AttackerModel, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# Load data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
trainset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

seen_size = int(0.8 * len(trainset))
unseen_size = len(trainset) - seen_size
seen_trainset, unseen_trainset = torch.utils.data.random_split(trainset, [seen_size, unseen_size])

seen_loader = torch.utils.data.DataLoader(seen_trainset, batch_size=64, shuffle=True)
unseen_train_loader = torch.utils.data.DataLoader(ConcatDataset([unseen_trainset, testset]), batch_size=64, shuffle=False)

baseline_model = CIFAR10Classifier().to(device)
baseline_model.load_state_dict(torch.load('/content/BaselineModel.pth'))
baseline_model.eval()
# Get the output size of the baseline model
with torch.no_grad():
    example_input = torch.randn(1, 3, 32, 32).to(device)
    baseline_output = baseline_model(example_input)
    attacker_baseline_input_size = baseline_output.size(1)

# Initialize the attacker model
num_shadow_models = 5
attacker_model = AttackerModel(attacker_baseline_input_size * num_shadow_models, 128, 1).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(attacker_model.parameters(), lr=0.001)
num_epochs = 5  # Increased epochs

# Initialize lists to store accuracies
train_accuracies_seen = []
train_accuracies_unseen = []
# Train multiple shadow models
num_shadow_models = 5
shadow_models = []
for i in range(num_shadow_models):
    shadow_model = CIFAR10Classifier().to(device)
    shadow_model.load_state_dict(torch.load(f'ShadowModel_{i}.pth'))
    shadow_model.eval()
    shadow_models.append(shadow_model)
# Train the attacker models using shadow models
for shadow_model in shadow_models:
  for epoch in range(num_epochs):
  # Evaluate the attacker model on seen data
        correct_seen, total_seen = 0, 0
        for seen_data, _ in seen_loader:
            seen_output = torch.cat([model(seen_data.to(device)) for model in shadow_models], dim=1)
            attacker_pred = attacker_model(seen_output)
            predicted_labels = (attacker_pred > 0.5).float()
            correct_seen += (predicted_labels == 1).sum().item()
            total_seen += seen_data.size(0)
        accuracy_seen = correct_seen / total_seen
        train_accuracies_seen.append(accuracy_seen)

        # Evaluate the attacker model on unseen data
        correct_unseen, total_unseen = 0, 0
        for unseen_data, _ in unseen_train_loader:
            unseen_output = torch.cat([model(unseen_data.to(device)) for model in shadow_models], dim=1)
            attacker_pred = attacker_model(unseen_output)
            predicted_labels = (attacker_pred > 0.5).float()
            correct_unseen += (predicted_labels == 0).sum().item()
            total_unseen += unseen_data.size(0)
        accuracy_unseen = correct_unseen / total_unseen
        train_accuracies_unseen.append(accuracy_unseen)

# Evaluate the attacker model
attacker_model.eval()
correct_seen, total_seen = 0, 0
correct_unseen, total_unseen = 0, 0

with torch.no_grad():
    for seen_data, _ in seen_loader:
        seen_output = torch.cat([model(seen_data.to(device)) for model in shadow_models], dim=1)
        attacker_pred = attacker_model(seen_output)
        predicted_labels = (attacker_pred > 0.5).float()
        correct_seen += (predicted_labels == 1).sum().item()
        total_seen += seen_data.size(0)

    for unseen_data, _ in unseen_train_loader:
        unseen_output = torch.cat([model(unseen_data.to(device)) for model in shadow_models], dim=1)
        attacker_pred = attacker_model(unseen_output)
        predicted_labels = (attacker_pred > 0.5).float()
        correct_unseen += (predicted_labels == 0).sum().item()
        total_unseen += unseen_data.size(0)

# Calculate accuracy
accuracy_seen = correct_seen / total_seen
accuracy_unseen = correct_unseen / total_unseen

print(f"Attacker Model Accuracy (Unseen Data): {accuracy_unseen:.4f}")


Files already downloaded and verified
Files already downloaded and verified
Attacker Model Accuracy (Unseen Data): 0.9850


## 3 Membership Inference Attack


### First Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import Subset, DataLoader, TensorDataset
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the pre-trained private model
model = CIFAR10Classifier()
state_dict = torch.load("model_state_dict.pth", map_location=device)
new_state_dict = {key.replace('_module.', ''): value for key, value in state_dict.items()}
model.load_state_dict(new_state_dict)
model.to(device)
model.eval()

# Load and preprocess the data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

DATA_ROOT = '../cifar10'
BATCH_SIZE = 64

# Load the indices from list.txt
indices_file = 'list.txt'
with open(indices_file, 'r') as f:
    indices = [int(line.strip()) for line in f]

full_train_dataset = CIFAR10(root=DATA_ROOT, train=True, download=True, transform=transform)
test_dataset = CIFAR10(root=DATA_ROOT, train=False, download=True, transform=transform)

train_indices_set = set(indices)
all_indices = set(range(len(full_train_dataset)))
other_indices = list(all_indices - train_indices_set)

train_dataset = Subset(full_train_dataset, indices[:len(indices)//2])
other_dataset = Subset(full_train_dataset, other_indices)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
other_loader = DataLoader(other_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Create labels
train_labels = torch.ones(len(train_dataset)).to(device)
other_labels = torch.zeros(len(other_dataset)).to(device)
test_labels = torch.zeros(len(test_dataset)).to(device)

def extract_features(model, dataloader):
    model.eval()
    features = []
    with torch.no_grad():
        for data in dataloader:
            inputs, _ = data
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs)
    return torch.cat(features).to(device)

# Extract features from the private model
train_features = extract_features(model, train_loader)
other_features = extract_features(model, other_loader)
test_features = extract_features(model, test_loader)

# Combine features and labels
combined_features = torch.cat((train_features, other_features, test_features))
combined_labels = torch.cat((train_labels, other_labels, test_labels))

# Create a new dataset and dataloader
new_dataset = TensorDataset(combined_features, combined_labels)
new_loader = DataLoader(new_dataset, batch_size=BATCH_SIZE, shuffle=True)
#########################################################################################
# Define the shadow model architecture
shadow = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, output_size)
)


attacker_model = nn.Sequential(
    nn.Linear(input_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, 10)  # Output 2 classes: member or non-member
)
attacker_optimizer = optim.Adam(attacker.parameters(), lr=0.001)
attacker_loss_fn = nn.BCELoss()

shadow_optimizer = optim.Adam(shadow.parameters(), lr=0.001)
shadow_loss_fn = nn.CrossEntropyLoss()

attacker.to(device)
shadow.to(device)

num_epochs = 10

for epoch in range(num_epochs):
    attacker.train()
    shadow.train()

    for features, labels in attacker_train_loader:
        features, labels = features.to(device), labels.to(device)

        # Train the attacker model
        attacker_optimizer.zero_grad()
        outputs = attacker(features).squeeze()
        loss = attacker_loss_fn(outputs, labels)
        loss.backward()
        attacker_optimizer.step()

        # Generate shadow model's predictions
        shadow_optimizer.zero_grad()
        shadow_outputs = shadow(features)
        shadow_loss = shadow_loss_fn(shadow_outputs, labels)
        shadow_loss.backward()
        shadow_optimizer.step()


# Evaluate the attacker model
attacker.eval()
all_labels = []
all_predicted = []
correct = 0
total = 0

with torch.no_grad():
    for features, labels in new_loader:
        features, labels = features.to(device), labels.to(device)
        outputs = attacker(features).squeeze()
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        all_labels.extend(labels.cpu().numpy())
        all_predicted.extend(predicted.cpu().numpy())

accuracy = correct / total
print(f'Training Accuracy: {accuracy:.4f}')

cm = confusion_matrix(all_labels, all_predicted)
precision = precision_score(all_labels, all_predicted)
recall = recall_score(all_labels, all_predicted)
f1 = f1_score(all_labels, all_predicted)

print(f'Confusion Matrix:\n{cm}')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


FileNotFoundError: [Errno 2] No such file or directory: 'list.txt'

### second Model

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import CIFAR10
from torchvision import transforms
from torch.utils.data import Subset, DataLoader, TensorDataset
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = CIFAR10Classifier()
state_dict = torch.load("model_state_dict.pth", map_location=device)
new_state_dict = {key.replace('_module.', ''): value for key, value in state_dict.items()}
model.load_state_dict(new_state_dict)
model.to(device)
model.eval()

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

DATA_ROOT = '../cifar10'
BATCH_SIZE = 64

# Load the indices from list.txt
indices_file = 'list.txt'
with open(indices_file, 'r') as f:
    indices = [int(line.strip()) for line in f]

full_train_dataset = CIFAR10(root=DATA_ROOT, train=True, download=True, transform=transform)
test_dataset = CIFAR10(root=DATA_ROOT, train=False, download=True, transform=transform)

train_indices_set = set(indices)
all_indices = set(range(len(full_train_dataset)))
other_indices = list(all_indices - train_indices_set)

train_dataset = Subset(full_train_dataset, indices[:len(indices)//2])
other_dataset = Subset(full_train_dataset, other_indices)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
other_loader = DataLoader(other_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Create labels
train_labels = torch.ones(len(train_dataset)).to(device)
other_labels = torch.zeros(len(other_dataset)).to(device)
test_labels = torch.zeros(len(test_dataset)).to(device)

def extract_features(model, dataloader):
    model.eval()
    features = []
    with torch.no_grad():
        for data in dataloader:
            inputs, _ = data
            inputs = inputs.to(device)
            outputs = model(inputs)
            features.append(outputs)
    return torch.cat(features).to(device)

train_features = extract_features(model, train_loader)
other_features = extract_features(model, other_loader)
test_features = extract_features(model, test_loader)

# Train shadow models
num_shadow_models = 5
shadow_models = []
for _ in range(num_shadow_models):
    shadow_model = LogisticRegression(random_state=42)
    shadow_model.fit(train_features.cpu(), train_labels.cpu())
    shadow_models.append(shadow_model)

# Membership inference attack
attacker = LogisticRegression(random_state=42)
all_features = torch.cat((train_features, other_features, test_features)).cpu()
all_labels = torch.cat((train_labels, other_labels, test_labels)).cpu()
train_data, val_data, train_labels, val_labels = train_test_split(all_features, all_labels, test_size=0.2, random_state=42)

# Train attacker model
attacker.fit(train_data, train_labels)

# Evaluate attacker model
attacker.eval()
val_predicted = attacker.predict(val_data)
accuracy = (val_labels == val_predicted).mean()
cm = confusion_matrix(val_labels, val_predicted)
precision = precision_score(val_labels, val_predicted)
recall = recall_score(val_labels, val_predicted)
f1 = f1_score(val_labels, val_predicted)

print(f'Attacker Model Accuracy: {accuracy:.4f}')
print(f'Confusion Matrix:\n{cm}')
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
